In [1]:
import spacy
import main
from tabulate import tabulate

nlp = main.load_spacy()

In [2]:
def explore(doc):
    result = [
         {"start": ent.start_char, "end": ent.end_char,
            "label": ent.label_, "id": ent.ent_id_, "text": ent.text}
        for ent in doc.ents
    ]
    print(result)

def emit(doc):
    print(doc)

def sentence(doc):
    for sent in doc.sents:
        print(sent.text)
def noun_chunks(doc):
    print("NOUN CHECKS - START")
    for chunk in doc.noun_chunks:
        print(chunk.text, chunk.root.text, chunk.root.dep_, chunk.root.head.text)
    print("NOUN CHECKS - END ")

def tokens_doc(doc):
    for token in doc:
        print(token.text, token.dep_, token.head.text, token.head.pos_, [child for child in token.children])

In [3]:
def extract_quantity(doc):
    print("Go")
    for ent in doc.ents:
        if "OBSERVATION"in ent.label_ :
            print("Found observation")
            head = ent.root.head
            rs = head.rights
            print(rs)
            
            

In [4]:

table_format = "psql" # plain, html, grid, github, 
def extract_observations(doc, debug=False):
        
    S_COLLECT_OBSERVATION = 0
    S_START_OBSERVATION = 1
    S_INIT = -1
    state = S_INIT 
    obs = None 
    obs_val = None 
    obs_unit = None 
    table = []
    observations = []
    for n in doc:
        table.append([n.text, n.ent_iob_, n.ent_type_ , n.ent_id_, n.pos_, n.tag_])
        ent = n.ent_iob_ 
        ent_type = n.ent_type_ 
        ent_id = n.ent_id_ 
        if "B" == ent and "OBSERVATION" == ent_type: 
            state = S_START_OBSERVATION
            if obs:
                if debug : 
                    print(f"Previous was {obs} = {obs_val} {obs_unit}") 
                observations.append([obs, obs_val, obs_unit])

            obs = ent_id
            
            obs_val = None
            obs_unit = None 
            if debug : 
                print(f"Start observation {obs}")
        elif "I" == ent: 
            state = S_START_OBSERVATION
        else:
            if state == S_START_OBSERVATION:
                state = S_COLLECT_OBSERVATION
                if debug: 
                    print(f"Collect obs {obs}")
                if "NUM" == n.pos_:
                    obs_val = n.text
                if "UNIT" == ent_type:
                    obs_unit = n.text

            elif state == S_COLLECT_OBSERVATION:
                if debug: 
                    print(f"Collect obs {obs}")
                if "NUM" == n.pos_: 
                    obs_val = n.text
                if "UNIT" == ent_type:
                    obs_unit = n.text

    observations.append([obs, obs_val, obs_unit])
    return [observations, table]


In [16]:
def write_results(result, PRINT_OBSERVATIONS=True, PRINT_NLP_TABLE=True):
    observations = result[0]
    table = result[1] 

    if PRINT_OBSERVATIONS: 
        print(tabulate(observations, headers=["ID", "VALUE", "UNIT"], tablefmt=table_format))

    if PRINT_NLP_TABLE: 
        print(tabulate(table, headers=["Text", "Ent", "Type", "Pos", "Speech", "Speech details"], tablefmt=table_format))



In [17]:

doc = nlp("Puls: 72 regelmessig. BP 120/80 mmHg. Temp 38 C. Resp. 45.  Smerte i venstre skulder. Redd for å dø. Ernæring må passes på.")
result = extract_observations(doc, False)
write_results(result, True, False) 

+----------------+---------+--------+
| ID             |   VALUE | UNIT   |
|----------------+---------+--------|
| PULSE          |      72 |        |
| BLOOD_PRESSURE |         | mmHg   |
| BODY_TEMP      |      38 |        |
| RESP           |      45 |        |
+----------------+---------+--------+


In [18]:
doc = nlp("Puls: 72 regelmessig. BP 120/80 mmHg. Temp 38 C. Resp. 45. ")
result = extract_observations(doc, False)
write_results(result, True, False) 

+----------------+---------+--------+
| ID             |   VALUE | UNIT   |
|----------------+---------+--------|
| PULSE          |      72 |        |
| BLOOD_PRESSURE |         | mmHg   |
| BODY_TEMP      |      38 |        |
| RESP           |      45 |        |
+----------------+---------+--------+


In [10]:
spacy.explain("PROPN")

'proper noun'